In [ ]:
import subprocess
import requests
import re
import os
from pathlib import Path

In [ ]:
working_folder = Path('.').resolve()
result = subprocess.run(["jupyter", "kernelspec", "list"], capture_output=True, text=True)
kernel_path = None
for line in result.stdout.splitlines():
    if 'python3' in line:
        parts = re.split(r'\s+', line.strip())
        if len(parts) > 1:
            kernel_path = Path(parts[1]).parent
            break
print(f"Install to {kernel_path}")

In [ ]:
dotnet_download_page = requests.get('https://dotnet.microsoft.com/download')
hrefs = re.findall(r'href="([^"]+)"', dotnet_download_page.text)

sdk_links = [href for href in hrefs if 'sdk' in href and 'windows-x64-installer' in href]
versions = []
for link in sdk_links:
    m = re.search(r'sdk-((\d+\.){1,2}\d+)-', link)
    if m:
        versions.append(m.group(1))

latest_ver = sorted(versions, key=lambda s: [int(x) for x in s.split('.')], reverse=True)[0]
latest_uri = None
for link in sdk_links:
    if f"sdk-{latest_ver}-windows-x64-installer" in link:
        latest_uri = 'https://dotnet.microsoft.com' + link
        break

installer_page = requests.get(latest_uri)
installer_hrefs = re.findall(r'href="([^"]+\.exe)"', installer_page.text)
file_uri = installer_hrefs[0]
exe_path = working_folder / 'dotnet.exe'
with requests.get(file_uri, stream=True) as r:
    r.raise_for_status()
    with open(exe_path, 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)

In [ ]:
subprocess.run([str(exe_path), '/install', '/passive', '/norestart'], check=True)

In [ ]:
exe_path.unlink()

In [ ]:
os.environ['PATH'] = os.environ.get('PATH', '')
proc_tool = subprocess.run(
    [
        'dotnet', 'tool', 'install', '-g', '--add-source',
        'https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-tools/nuget/v3/index.json', 'Microsoft.dotnet-interactive'
    ],
    capture_output=True,
    text=True
)
print(proc_tool.stdout)
print(proc_tool.stderr)

In [ ]:
proc_jupyter = subprocess.run(
    ['dotnet', 'interactive', 'jupyter', 'install', '--path', str(kernel_path)],
    capture_output=True,
    text=True
)
print(proc_jupyter.stdout)
print(proc_jupyter.stderr)